# Generalizing a Taylor Recurrence

In [2]:
from sumpy.recurrence import _make_sympy_vec

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_off_axis_expression, get_reindexed_and_center_origin_off_axis_recurrence, _extract_idx_terms_from_recurrence

import sympy as sp
from sympy import hankel1

import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import cm, ticker

In [3]:
var = _make_sympy_vec("x", 2)
s = sp.Function("s")
n = sp.symbols("n")

In [4]:
from sumpy.expansion.diff_op import laplacian,make_identity_diff_op
w = make_identity_diff_op(2)
laplace2d = laplacian(w)
helmholtz2d = laplacian(w) + w

In [5]:
var = _make_sympy_vec("x", 2)
var_t = _make_sympy_vec("t", 2)
g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
derivs_laplace = [g_x_y.subs(var_t[0], 0).subs(var_t[1], 0).diff(var[0], i)
                    for i in range(8)]

In [23]:
i=4
j=4
derivs_laplace[i].diff(var[0], j).subs(var[0], 0)

-5040/x1**8

In [ ]:
k = 1
abs_dist = sp.sqrt((var[0]-var_t[0])**2 +
                       (var[1]-var_t[1])**2)
g_x_y = (1j/4) * hankel1(0, k * abs_dist)
derivs_helmholtz = [sp.diff(g_x_y,
                    var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                                            for i in range(6)]

In [1]:
derivs_helmholtz[4].diff(var[1], 0).subs(var[1], 0)

NameError: name 'derivs_helmholtz' is not defined